In [39]:
#knn.py
import csv
import random
import math
import operator
from sklearn.preprocessing import MinMaxScaler

def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    print('loadDataset')
    azki = pd.read_csv('churn_balanced.csv')
    dataset = azki.dropna().values.tolist()
    dataset2 = dataset
    scaler = MinMaxScaler()
    scaler.fit(dataset)
    MinMaxScaler(copy=True, feature_range=(0, 1))
    dataset = scaler.transform(dataset)
    for a in range(len(dataset)):
        dataset[a][19] = dataset2[a][19]
    for x in range(0,len(dataset)):
        for y in range(19):
            dataset[x][y] = float(dataset[x][y])
        if random.random() < split:
            trainingSet.append(dataset[x])
        else:
            testSet.append(dataset[x])

def euclideanDistance(instance1, instance2, length, chrom2):
    distance = 0
    for x in range(1,length):
        if chrom2[x]==1:
            distance += pow((float(instance1[x])-float(instance2[x])), 2)
    return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k, chrom2):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length, chrom2)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
    print('getAccuracy')
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def fitnessValue(chrom):
    print('fitnessValue')
    random.seed(2)
    trainingSet=[]
    testSet=[]
    split = 0.8
    loadDataset('churn_numeric2.csv', split, trainingSet, testSet)
    predictions=[]
    k = 1
    for x in range(len(testSet)):
        if x%50==0:
            print('.',end='')
        neighbors = getNeighbors(trainingSet, testSet[x], k, chrom)
        result = getResponse(neighbors)
        predictions.append(result)
    accuracy = getAccuracy(testSet, predictions)
    return repr(accuracy)

#ga.py


In [40]:
import numpy
import pandas as pd

In [41]:
def generatePop(n):
    print('generatePop')
    chromAsli=[]
    for x in range(n):
        temp=[]
        temp2=[]
        for x in range(19):
            temp.append(numpy.random.randint(1))
        temp2.append(temp)
        fitness=fitnessValue(temp)
        temp2.append(fitness)
        chromAsli.append(temp2)
    return chromAsli

def eliteChild(chrom2,n):
    print('eliteChild')
    a=sorted(chrom2,key=lambda l:l[1], reverse=True)
    bestVal=a[0][1]
    bestFt2=a[0][0]
    next2=[]
    chrom3=[]
    for x in range(2):
        next2.append(a[x][0])
    for x in range(2,n):
        chrom3.append(a[x])
    return chrom3,next2,bestVal,bestFt2

def tournament(chrom2):
    print('tournament')
    best=[]
    panjang=len(chrom2)-1
    for x in range(2):
        a=numpy.random.randint(0,panjang)
        best.append(chrom2[a])
    bestOne=best[0]
    if(best[0][1]<best[1][1]):
        bestOne=best[1]
    return bestOne[0]

def getMutation(chrom2,pm,next2,n):
    print("getMutation") 
    rng=n-len(next2)
    for i in range(rng):
        a=tournament(chrom2)
        rd=[]
        for x in range(21):
            ru=numpy.random.uniform(0.0, 1.0)
            if ru<=pm:
                rd.append(1) 
            else:
                rd.append(0)
        if a==rd:
            a=tournament(chrom2)
        result=toXor(a,rd)
        next2.append(result)
    return next2

def toXor(x1,x2):
    print('toXor')
    xorRes=[]
    print(x1,x2)
    for x in range(len(x1)):
        if x1[x]==x2[x]:
            xorRes.append(0)
        else:
            xorRes.append(1)
    return xorRes

def getCrossOver(chrom2,pr,next2):
    print('getCrossOver')
    rng=int(pr*len(chrom2))
    for x in range(rng):
        a=tournament(chrom2)
        b=tournament(chrom2)
        if a==b:
            result=a
        else:
            result=toXor(a,b)
        next2.append(result)
    return next2

def getGeneration(chrom2,idx):
    print('getGeneration')
    chrom2,nextGen,bestFitness,bestFt2=eliteChild(chrom2,genNum)
    nextGen=getCrossOver(chrom2,0.8,nextGen)
    nextGen=getMutation(chrom2,0.3,nextGen,genNum)
    print('Generation {} {:.3f}'.format(idx+1,float(bestFitness)),end='%\n')
    return nextGen,bestFitness,bestFt2

genNum=6
knn_chrom=[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
knn_acc=fitnessValue(knn_chrom)
limit=10
stall=3
chrom=generatePop(genNum)
counter=0
newFit=0.0
oldFit=0.0
print("kerja")
for x in range(limit):
    print("round", x)
    oldFit=newFit
    newChrom,newFit,bestFt=getGeneration(chrom,x)
    newFit=float(newFit)
    temp2=[]
    for y in range(genNum):
        print("gen", y)
        temp=[]
        temp.append(newChrom[y])
        f=fitnessValue(newChrom[y])
        temp.append(f)
        temp2.append(temp)
    chrom=temp2
    diff=newFit - oldFit
    if diff<=0.0001:
        counter+=1
    else:
        counter=0
    if counter==stall:
        break
    
print("Accuracy using K-NN without GA: ",end='')
print('{:.3f}'.format(float(knn_acc)),end='')
print("%\nAccuracy using K-NN with GA:",end='')
print('{:.3f}'.format(float(newFit)),end='%')
print("\nUsed features:", bestFt)

fitnessValue
loadDataset
.........................................getAccuracy
generatePop
fitnessValue
loadDataset
.........................................getAccuracy
fitnessValue
loadDataset
.........................................getAccuracy
fitnessValue
loadDataset
.........................................getAccuracy
fitnessValue
loadDataset
.........................................getAccuracy
fitnessValue
loadDataset
.........................................getAccuracy
fitnessValue
loadDataset
.........................................getAccuracy
kerja
round 0
getGeneration
eliteChild
getCrossOver
tournament
tournament
tournament
tournament
tournament
tournament
getMutation
tournament
toXor
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0]
Generation 1 50.074%
gen 0
fitnessValue
loadDataset
.........................................getAccuracy
gen 1
fitnessValue
loadDataset
.....................................